In [1]:
import pandas as pd
import numpy as np
from ydata_profiling import ProfileReport
import re
# import nltk
from wordcloud import STOPWORDS

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score

In [2]:
DATASET_PATH = r"dataset/News_Category_Dataset_v3.json"



In [3]:
df = pd.read_json(DATASET_PATH, lines=True)
df.head()

,link,headline,category,short_description,authors,date
0,https://www.huffpost.com/entry/covid-boosters-...,Over 4 Million Americans Roll Up Sleeves For O...,U.S. NEWS,Health experts said it is too early to predict...,"Carla K. Johnson, AP",2022-09-23
1,https://www.huffpost.com/entry/american-airlin...,"American Airlines Flyer Charged, Banned For Li...",U.S. NEWS,He was subdued by passengers and crew when he ...,Mary Papenfuss,2022-09-23
2,https://www.huffpost.com/entry/funniest-tweets...,23 Of The Funniest Tweets About Cats And Dogs ...,COMEDY,"""Until you have a dog you don't understand wha...",Elyse Wanshel,2022-09-23
3,https://www.huffpost.com/entry/funniest-parent...,The Funniest Tweets From Parents This Week (Se...,PARENTING,"""Accidentally put grown-up toothpaste on my to...",Caroline Bologna,2022-09-23
4,https://www.huffpost.com/entry/amy-cooper-lose...,Woman Who Called Cops On Black Bird-Watcher Lo...,U.S. NEWS,Amy Cooper accused investment firm Franklin Te...,Nina Golgowski,2022-09-22


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 209527 entries, 0 to 209526
Data columns (total 6 columns):
 #   Column             Non-Null Count   Dtype         
---  ------             --------------   -----         
 0   link               209527 non-null  object        
 1   headline           209527 non-null  object        
 2   category           209527 non-null  object        
 3   short_description  209527 non-null  object        
 4   authors            209527 non-null  object        
 5   date               209527 non-null  datetime64[ns]
dtypes: datetime64[ns](1), object(5)
memory usage: 9.6+ MB


In [5]:
df.describe(include='all')

C:\Users\Shridhar\AppData\Local\Temp\ipykernel_8712\2884002236.py:1: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  df.describe(include='all')


,link,headline,category,short_description,authors,date
count,209527,209527,209527,209527,209527,209527
unique,209486,207996,42,187022,29169,3890
top,https://www.huffingtonpost.comhttps://www.wash...,Sunday Roundup,POLITICS,,,2014-03-25 00:00:00
freq,2,90,35602,19712,37418,100
first,NaN,NaN,NaN,NaN,NaN,2012-01-28 00:00:00
last,NaN,NaN,NaN,NaN,NaN,2022-09-23 00:00:00


In [6]:
df.describe(include='all').style.set_properties(**{"background-color":"#00539C",
                                                "color":"#EEA47F"})

C:\Users\Shridhar\AppData\Local\Temp\ipykernel_8712\2676857340.py:1: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  df.describe(include='all').style.set_properties(**{"background-color":"#00539C",


,link,headline,category,short_description,authors,date
count,209527,209527,209527,209527,209527,209527
unique,209486,207996,42,187022,29169,3890
top,https://www.huffingtonpost.comhttps://www.washingtonpost.com/politics/divisions-within-gop-over-trumps-candidacy-are-growing/2016/02/28/97b16010-de3a-11e5-8d98-4b3d9215ade1_story.html,Sunday Roundup,POLITICS,,,2014-03-25 00:00:00
freq,2,90,35602,19712,37418,100
first,nan,nan,nan,nan,nan,2012-01-28 00:00:00
last,nan,nan,nan,nan,nan,2022-09-23 00:00:00


In [7]:
ProfileReport(df)

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

ValueError: Couldn't find space to draw. Either the Canvas size is too small or too much of the image is masked out.

In [8]:
df.isna().sum()

link                 0
headline             0
category             0
short_description    0
authors              0
date                 0
dtype: int64

In [9]:
df.head()

,link,headline,category,short_description,authors,date
0,https://www.huffpost.com/entry/covid-boosters-...,Over 4 Million Americans Roll Up Sleeves For O...,U.S. NEWS,Health experts said it is too early to predict...,"Carla K. Johnson, AP",2022-09-23
1,https://www.huffpost.com/entry/american-airlin...,"American Airlines Flyer Charged, Banned For Li...",U.S. NEWS,He was subdued by passengers and crew when he ...,Mary Papenfuss,2022-09-23
2,https://www.huffpost.com/entry/funniest-tweets...,23 Of The Funniest Tweets About Cats And Dogs ...,COMEDY,"""Until you have a dog you don't understand wha...",Elyse Wanshel,2022-09-23
3,https://www.huffpost.com/entry/funniest-parent...,The Funniest Tweets From Parents This Week (Se...,PARENTING,"""Accidentally put grown-up toothpaste on my to...",Caroline Bologna,2022-09-23
4,https://www.huffpost.com/entry/amy-cooper-lose...,Woman Who Called Cops On Black Bird-Watcher Lo...,U.S. NEWS,Amy Cooper accused investment firm Franklin Te...,Nina Golgowski,2022-09-22


In [10]:
df.category.value_counts()

POLITICS          35602
WELLNESS          17945
ENTERTAINMENT     17362
TRAVEL             9900
STYLE & BEAUTY     9814
PARENTING          8791
HEALTHY LIVING     6694
QUEER VOICES       6347
FOOD & DRINK       6340
BUSINESS           5992
COMEDY             5400
SPORTS             5077
BLACK VOICES       4583
HOME & LIVING      4320
PARENTS            3955
THE WORLDPOST      3664
WEDDINGS           3653
WOMEN              3572
CRIME              3562
IMPACT             3484
DIVORCE            3426
WORLD NEWS         3299
MEDIA              2944
WEIRD NEWS         2777
GREEN              2622
WORLDPOST          2579
RELIGION           2577
STYLE              2254
SCIENCE            2206
TECH               2104
TASTE              2096
MONEY              1756
ARTS               1509
ENVIRONMENT        1444
FIFTY              1401
GOOD NEWS          1398
U.S. NEWS          1377
ARTS & CULTURE     1339
COLLEGE            1144
LATINO VOICES      1130
CULTURE & ARTS     1074
EDUCATION       

In [11]:
df.category.nunique()

42

In [12]:
cat_df = pd.DataFrame(df.category.value_counts())
cat_df.reset_index()


,index,category
0,POLITICS,35602
1,WELLNESS,17945
2,ENTERTAINMENT,17362
3,TRAVEL,9900
4,STYLE & BEAUTY,9814
5,PARENTING,8791
6,HEALTHY LIVING,6694
7,QUEER VOICES,6347
8,FOOD & DRINK,6340
9,BUSINESS,5992


In [13]:
cat_df["percetage"] = [count/ cat_df["count"].sum() *100 for count in cat_df["count"]]
cat_df

KeyError: 'count'

In [14]:
df.head()

,link,headline,category,short_description,authors,date
0,https://www.huffpost.com/entry/covid-boosters-...,Over 4 Million Americans Roll Up Sleeves For O...,U.S. NEWS,Health experts said it is too early to predict...,"Carla K. Johnson, AP",2022-09-23
1,https://www.huffpost.com/entry/american-airlin...,"American Airlines Flyer Charged, Banned For Li...",U.S. NEWS,He was subdued by passengers and crew when he ...,Mary Papenfuss,2022-09-23
2,https://www.huffpost.com/entry/funniest-tweets...,23 Of The Funniest Tweets About Cats And Dogs ...,COMEDY,"""Until you have a dog you don't understand wha...",Elyse Wanshel,2022-09-23
3,https://www.huffpost.com/entry/funniest-parent...,The Funniest Tweets From Parents This Week (Se...,PARENTING,"""Accidentally put grown-up toothpaste on my to...",Caroline Bologna,2022-09-23
4,https://www.huffpost.com/entry/amy-cooper-lose...,Woman Who Called Cops On Black Bird-Watcher Lo...,U.S. NEWS,Amy Cooper accused investment firm Franklin Te...,Nina Golgowski,2022-09-22


In [15]:
data= df.copy()

In [16]:
data["news_combine_content"] = data["headline"] + data["short_description"]
data.head(2)

,link,headline,category,short_description,authors,date,news_combine_content
0,https://www.huffpost.com/entry/covid-boosters-...,Over 4 Million Americans Roll Up Sleeves For O...,U.S. NEWS,Health experts said it is too early to predict...,"Carla K. Johnson, AP",2022-09-23,Over 4 Million Americans Roll Up Sleeves For O...
1,https://www.huffpost.com/entry/american-airlin...,"American Airlines Flyer Charged, Banned For Li...",U.S. NEWS,He was subdued by passengers and crew when he ...,Mary Papenfuss,2022-09-23,"American Airlines Flyer Charged, Banned For Li..."


In [17]:
data["length_count"] = data["news_combine_content"].apply(lambda x : len(x))
data.head()

,link,headline,category,short_description,authors,date,news_combine_content,length_count
0,https://www.huffpost.com/entry/covid-boosters-...,Over 4 Million Americans Roll Up Sleeves For O...,U.S. NEWS,Health experts said it is too early to predict...,"Carla K. Johnson, AP",2022-09-23,Over 4 Million Americans Roll Up Sleeves For O...,230
1,https://www.huffpost.com/entry/american-airlin...,"American Airlines Flyer Charged, Banned For Li...",U.S. NEWS,He was subdued by passengers and crew when he ...,Mary Papenfuss,2022-09-23,"American Airlines Flyer Charged, Banned For Li...",248
2,https://www.huffpost.com/entry/funniest-tweets...,23 Of The Funniest Tweets About Cats And Dogs ...,COMEDY,"""Until you have a dog you don't understand wha...",Elyse Wanshel,2022-09-23,23 Of The Funniest Tweets About Cats And Dogs ...,133
3,https://www.huffpost.com/entry/funniest-parent...,The Funniest Tweets From Parents This Week (Se...,PARENTING,"""Accidentally put grown-up toothpaste on my to...",Caroline Bologna,2022-09-23,The Funniest Tweets From Parents This Week (Se...,215
4,https://www.huffpost.com/entry/amy-cooper-lose...,Woman Who Called Cops On Black Bird-Watcher Lo...,U.S. NEWS,Amy Cooper accused investment firm Franklin Te...,Nina Golgowski,2022-09-22,Woman Who Called Cops On Black Bird-Watcher Lo...,233


In [18]:
data["length_count"].describe()

count    209527.000000
mean        172.624025
std          78.552970
min           0.000000
25%         122.000000
50%         170.000000
75%         207.000000
max        1486.000000
Name: length_count, dtype: float64

In [19]:
data[data["length_count"] < 20]

,link,headline,category,short_description,authors,date,news_combine_content,length_count
20773,https://www.huffingtonpost.com/entry/the-big-b...,The Big Blue Wave,COMEDY,,"Shan Wells, ContributorSciency sun venerator +...",2017-11-10,The Big Blue Wave,17
34488,https://www.huffingtonpost.com/entry/the-bechd...,The Bechdel Test,COMEDY,,"Hilary Fitzgerald Campbell, ContributorHilary'...",2017-05-24,The Bechdel Test,16
43388,https://www.huffingtonpost.com/entry/health-ca...,Health Care Reform,POLITICS,,"Darryl S. Weiman, M.D., J.D., ContributorProfe...",2017-02-11,Health Care Reform,18
46509,https://www.huffingtonpost.com/entry/home-for-...,Home for Christmas,IMPACT,,"Shad Meshad, ContributorFounder and co-author,...",2017-01-08,Home for Christmas,18
46515,https://www.huffingtonpost.com/entry/the-last-...,The Last Broadcast,ENTERTAINMENT,,"Alice Carey, ContributorWriter and style icon",2017-01-08,The Last Broadcast,18
...,...,...,...,...,...,...,...,...
178896,https://www.huffingtonpost.com/entry/the-end-i...,The End Is Near,COMEDY,,"Chris Michael, Contributor\nContributor",2012-12-23,The End Is Near,15
181760,https://www.huffingtonpost.com/entry/gay-marri...,Gay Marriage,COMEDY,,"Katie Goodman, Contributor\nAward-winning comi...",2012-11-22,Gay Marriage,12
183903,https://www.huffingtonpost.com/entry/gosling-a...,Gosling and Booker,COMEDY,,"Corey Pandolph, Contributor\nCartoonist for Th...",2012-10-30,Gosling and Booker,18
193052,https://www.huffingtonpost.com/entry/haute-dad...,Haute Dads,STYLE & BEAUTY,,"Mara Menachem, Contributor\nSelf-effacing, aut...",2012-07-23,Haute Dads,10


In [20]:
final = data.copy()

In [21]:
final = final[["news_combine_content","category"]]
final.head(2)

,news_combine_content,category
0,Over 4 Million Americans Roll Up Sleeves For O...,U.S. NEWS
1,"American Airlines Flyer Charged, Banned For Li...",U.S. NEWS


In [22]:
def clean_text(text):
    text = text.lower()
    text = re.sub(r'[^a-zA-Z]',' ', text)
    text = text.strip()
    
    # remove stop words
    text = [word for word in text.split() if word not in list(STOPWORDS)]
    
    text = " ".join(text)
    return text
    

In [23]:
clean_text(final["news_combine_content"][0])

'million americans roll sleeves omicron targeted covid boostershealth experts said early predict whether demand match million doses new boosters u s ordered fall'

In [24]:
final["clean_text"] = final["news_combine_content"].apply(clean_text)

In [25]:
final.head(2)

,news_combine_content,category,clean_text
0,Over 4 Million Americans Roll Up Sleeves For O...,U.S. NEWS,million americans roll sleeves omicron targete...
1,"American Airlines Flyer Charged, Banned For Li...",U.S. NEWS,american airlines flyer charged banned life pu...


In [56]:
x = final["clean_text"]
y = final["category"]

In [27]:
### NLP With ML 

In [28]:
count_vect = CountVectorizer()
x_count_vect = count_vect.fit_transform(x)

In [29]:
import pickle
with open("vector.pkl","wb") as file:
    pickle.dump(count_vect,file)

In [31]:
y.unique()

array(['U.S. NEWS', 'COMEDY', 'PARENTING', 'WORLD NEWS', 'CULTURE & ARTS',
       'TECH', 'SPORTS', 'ENTERTAINMENT', 'POLITICS', 'WEIRD NEWS',
       'ENVIRONMENT', 'EDUCATION', 'CRIME', 'SCIENCE', 'WELLNESS',
       'BUSINESS', 'STYLE & BEAUTY', 'FOOD & DRINK', 'MEDIA',
       'QUEER VOICES', 'HOME & LIVING', 'WOMEN', 'BLACK VOICES', 'TRAVEL',
       'MONEY', 'RELIGION', 'LATINO VOICES', 'IMPACT', 'WEDDINGS',
       'COLLEGE', 'PARENTS', 'ARTS & CULTURE', 'STYLE', 'GREEN', 'TASTE',
       'HEALTHY LIVING', 'THE WORLDPOST', 'GOOD NEWS', 'WORLDPOST',
       'FIFTY', 'ARTS', 'DIVORCE'], dtype=object)

In [90]:
tfidf_vect = TfidfVectorizer()
x_tfidf_vect = tfidf_vect.fit_transform(x)

In [92]:
x_train,x_test, y_train,y_test =train_test_split(x_count_vect,y, test_size=0.2, random_state=11)

In [93]:
model_count = MultinomialNB()
model_count.fit(x_train,y_train)

MultinomialNB()

In [94]:
train_acc = model_count.score(x_train,y_train)
print(f"Train Accuracy = {train_acc}")

test_acc = model_count.score(x_test,y_test)
print(f"Test Acciracy = {test_acc}")

Train Accuracy = 0.5598522858114436
Test Acciracy = 0.49033551281439414


In [95]:
x_train,x_test, y_train,y_test =train_test_split(x_tfidf_vect,y, test_size=0.2, random_state=11)
model_tfidf = MultinomialNB()
model_tfidf.fit(x_train,y_train)

MultinomialNB()

In [96]:
train_acc = model_tfidf.score(x_train,y_train)
print(f"Train Accuracy = {train_acc}")

test_acc = model_tfidf.score(x_test,y_test)
print(f"Test Acciracy = {test_acc}")

Train Accuracy = 0.3917170282959772
Test Acciracy = 0.3654607932038372


In [ ]:
### NLP with DL 

In [33]:
from keras.callbacks import ModelCheckpoint
from keras.preprocessing.text import Tokenizer
from keras.utils import to_categorical, pad_sequences
from keras.callbacks import EarlyStopping

from keras.layers import Embedding, Flatten, Dense, Dropout
from keras.models import Sequential
from keras.layers import SimpleRNN, Bidirectional,GlobalMaxPool1D, MaxPool1D
from keras.regularizers import L1L2
from sklearn.preprocessing import LabelEncoder

In [57]:
# Tokenize the data 
encoder = LabelEncoder()
y = encoder.fit_transform(y)

## Spilting the data 
x_train,x_test,y_train,y_test = train_test_split(x,y, test_size=0.20)

tokenizer = Tokenizer(num_words=100000, oov_token = True)
tokenizer.fit_on_texts(x_train)

## Padding the text input training
train_seq = tokenizer.texts_to_sequences(x_train)
train_padding_seq = pad_sequences(train_seq,maxlen=130)

## Padding the text input testing 
test_seq = tokenizer.texts_to_sequences(x_test)
test_padding_seq = pad_sequences(test_seq,maxlen=130)

### Output 

y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

## Embedding word index 

word_index = tokenizer.word_index
max_words = 150000
total_words = len(word_index)

In [38]:

with open("tokenizer.pkl","wb") as file:
        pickle.dump(tokenizer,file)

In [35]:
train_padding_seq.shape

(167621, 130)

In [42]:
## Model 

model = Sequential()
model.add(Embedding(total_words,70, input_length=130))
model.add(Bidirectional(SimpleRNN(64, dropout=0.1,recurrent_dropout=0.20, activation='tanh', return_sequences=True)))
model.add(Bidirectional(SimpleRNN(64, dropout=0.1,recurrent_dropout=0.30, activation='tanh', return_sequences=True)))
model.add(SimpleRNN(32, activation='tanh'))
model.add(Dropout(0.2))
model.add(Dense(42,activation='softmax'))
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 130, 70)           11102840  
                                                                 
 bidirectional (Bidirectiona  (None, 130, 128)         17280     
 l)                                                              
                                                                 
 bidirectional_1 (Bidirectio  (None, 130, 128)         24704     
 nal)                                                            
                                                                 
 simple_rnn_2 (SimpleRNN)    (None, 32)                5152      
                                                                 
 dropout (Dropout)           (None, 32)                0         
                                                                 
 dense (Dense)               (None, 42)                1

In [121]:
# Model Compile
model.compile(optimizer='rmsprop',
              loss="categorical_crossentropy", 
              metrics=['accuracy']
             )
# model Early stoppping
earlystopping = EarlyStopping(monitor= 'accuracy',
                                              patience=5, 
                                              verbose=1,
                                              mode='min'
                                             )
checkpointer = ModelCheckpoint(filepath='bestvalue', 
                               monitor='val_loss', 
                               verbose=0,
                               save_best_only=True)
callback_list = [checkpointer,earlystopping]

# fit model on the data 
result = model.fit(train_padding_seq,y_train, 
                  batch_size=128,
                  epochs=2,
                  validation_split=0.2)

# evaluate the model 
test_loss, test_acc = model.evaluate(test_padding_seq, y_test, verbose=0)
print(f"Test Loss = {test_loss}\nTest Accuracy = {test_acc}")



Epoch 1/2
1048/1048 [==============================] - 780s 738ms/step - loss: 3.1914 - accuracy: 0.1993 - val_loss: 2.8060 - val_accuracy: 0.2731
Epoch 2/2
1048/1048 [==============================] - 833s 795ms/step - loss: 2.6455 - accuracy: 0.3285 - val_loss: 2.4116 - val_accuracy: 0.3720
Test Loss = 2.420158624649048
Test Accuracy = 0.37359803915023804


In [122]:
!mkdir saved_model
model.save(r"saved_model/simpleRNN_v1")

INFO:tensorflow:Assets written to: saved_model/simpleRNN\assets


INFO:tensorflow:Assets written to: saved_model/simpleRNN\assets


In [ ]:
tensorflowl_obj =keras.models.load_model(r"saved_model/simpleRNN")
model_obj.summary()

In [ ]:
text = """We live in a world of too much information and too little context. Too much noise and too little insight. That's where TOI's explainers come in. """

clean_text = clean_text(text)
text_seq = tokenizerenizer.text_to_sequence(clean_text)
text_padpad_sequences= pad_sequences(text_seq, maxlen=130)

result_predict = model_obj.predict(text_padseq)
result_predict

In [123]:
import pickle 



In [124]:
with open("ml_model.pkl","wb") as file:
    pickle.dump(model_count,file)

In [46]:
news = """Reports of breach of data from CoWin without any basis: Health ministry
The Union health ministry on Monday rejected several media reports about a massive data breach in the CoWin platform, even as it requested the country's nodal cyber security agency CERT-In to look into the issue and submit a report.
GenZ carries a spirit of unyielding freedom. They are the young & fearless ones, always on the move to explore the unknown. They are expressive & so want their gadgets to help them step up in life. Understanding this pulse of the GenZ is the tech major Samsung. The brand has carved out a special place amongst the youth as a dependable smartphone maker who always has their back. And today, Samsung has launched a device that aims to exceed their expectations. Make way for Samsung Galaxy F54 5G - the latest offering from Samsung's camp with much to offer
"""
print(f"Orginal content\n= {news}")
clean_news = clean_text(news)

print(f"After Cleaning\n= {clean_news}")

tokenizer = Tokenizer(num_words=100000, oov_token = True)
tokenizer.fit_on_texts(x_train)

text_seq = tokenizer.texts_to_sequences(clean_news)
text_padpad_sequences= pad_sequences(text_seq, maxlen=130)

Orginal content
= Reports of breach of data from CoWin without any basis: Health ministry
The Union health ministry on Monday rejected several media reports about a massive data breach in the CoWin platform, even as it requested the country's nodal cyber security agency CERT-In to look into the issue and submit a report.
GenZ carries a spirit of unyielding freedom. They are the young & fearless ones, always on the move to explore the unknown. They are expressive & so want their gadgets to help them step up in life. Understanding this pulse of the GenZ is the tech major Samsung. The brand has carved out a special place amongst the youth as a dependable smartphone maker who always has their back. And today, Samsung has launched a device that aims to exceed their expectations. Make way for Samsung Galaxy F54 5G - the latest offering from Samsung's camp with much to offer

After Cleaning
= reports breach data cowin without basis health ministry union health ministry monday rejected several

In [51]:
encoder

LabelEncoder()

In [58]:
encoder.classes_

array(['ARTS', 'ARTS & CULTURE', 'BLACK VOICES', 'BUSINESS', 'COLLEGE',
       'COMEDY', 'CRIME', 'CULTURE & ARTS', 'DIVORCE', 'EDUCATION',
       'ENTERTAINMENT', 'ENVIRONMENT', 'FIFTY', 'FOOD & DRINK',
       'GOOD NEWS', 'GREEN', 'HEALTHY LIVING', 'HOME & LIVING', 'IMPACT',
       'LATINO VOICES', 'MEDIA', 'MONEY', 'PARENTING', 'PARENTS',
       'POLITICS', 'QUEER VOICES', 'RELIGION', 'SCIENCE', 'SPORTS',
       'STYLE', 'STYLE & BEAUTY', 'TASTE', 'TECH', 'THE WORLDPOST',
       'TRAVEL', 'U.S. NEWS', 'WEDDINGS', 'WEIRD NEWS', 'WELLNESS',
       'WOMEN', 'WORLD NEWS', 'WORLDPOST'], dtype=object)

In [60]:
encoder.inverse_transform([16])

array(['HEALTHY LIVING'], dtype=object)

In [55]:
y

array([35, 35,  5, ..., 28, 28, 28], dtype=int64)

In [61]:
with open("labels.pkl","wb") as file: 
    pickle.dump(encoder,file)